In [21]:
from keras.models import load_model, model_from_json
import pandas as pd
import numpy as np
import os
import re

In [22]:
# model = load_model('ner_elmo_bilstm.h5')
# model.load_weights('ner_elmo_bilstm_weights.h5')
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")

In [23]:
from allennlp.modules.elmo import Elmo,batch_to_ids

In [50]:
sentence = "उत्तराखंड की राजधानी देहरादून में फरवरी की दुपहरी पिछले दस साल में सबसे गर्म रही".split()
# sentence = "अरे सारथी जम्मू कैसा मौसम है".split()
# sentence = "मुझे बताओ हांगकांग में तापमान क्या है".split()

In [51]:
X = [w for w in sentence]
X

['उत्तराखंड',
 'की',
 'राजधानी',
 'देहरादून',
 'में',
 'फरवरी',
 'की',
 'दुपहरी',
 'पिछले',
 'दस',
 'साल',
 'में',
 'सबसे',
 'गर्म',
 'रही']

In [52]:
max_len = 16

In [53]:
if len(X)<max_len:
    while len(X)<max_len:
        X += ["__PAD__"]
X = [X]

In [28]:
datadir = os.path.join('hi', 'elmo')
options_file = os.path.join(datadir, 'hi-d512-options.json')
weight_file = os.path.join(datadir, 'hi-d512-elmo.hdf5')

elmo = Elmo(options_file,weight_file,1,dropout=0)

In [54]:
embeds = elmo(batch_to_ids(X))
embedding = embeds["elmo_representations"]

In [55]:
import torch
import tensorflow as tf

embedding1 = []
for t in embedding:
    np_tensor = t.detach().numpy()
    embedding1 += [np_tensor.tolist()]

embedding = embedding1

In [56]:
# embedding

In [57]:
p = model.predict(np.array(embedding).reshape(1,16,1024))
p = np.argmax(p, axis=-1)

In [58]:
p

array([[8, 1, 8, 8, 1, 0, 1, 1, 0, 1, 3, 1, 1, 9, 1, 1]], dtype=int64)

In [59]:
df = pd.DataFrame(columns=['Sentence #','Word','Tag'])

In [60]:
data_dir = r'Data/'

In [61]:
i=1
data = list()
with open(data_dir+'nertrnweather.txt',encoding='utf-8') as f:
    for line in f.readlines():
        if line=='\n':
            i+=1
        else:
            data=line.split(" ")
            df=df.append({"Sentence #":f"Sentence {i}","Word":data[0],"Tag":re.sub("\n","",data[1])},ignore_index=True)
df

,Sentence #,Word,Tag
0,Sentence 1,अरे,O
1,Sentence 1,सारथी,O
2,Sentence 1,जम्मू,U-location
3,Sentence 1,कैसा,O
4,Sentence 1,मौसम,O
...,...,...,...
1403,Sentence 203,की,O
1404,Sentence 203,स्थिति,O
1405,Sentence 203,क्या,O
1406,Sentence 203,है,O


In [62]:
# tags = list(set(df["Tag"].values))
# n_tags = len(tags); n_tags
tags = ['B-date',
 'O',
 'B-weather_type',
 'L-date',
 'L-weather_type',
 'B-location',
 'U-date',
 'L-location',
 'U-location',
 'U-weather_type']

In [63]:
p.tolist()

[[8, 1, 8, 8, 1, 0, 1, 1, 0, 1, 3, 1, 1, 9, 1, 1]]

In [64]:
for w,pred in zip(X[0],p.tolist()[0]):
    print(w,tags[pred])

उत्तराखंड U-location
की O
राजधानी U-location
देहरादून U-location
में O
फरवरी B-date
की O
दुपहरी O
पिछले B-date
दस O
साल L-date
में O
सबसे O
गर्म U-weather_type
रही O
__PAD__ O


In [46]:
tags

['B-location',
 'U-weather_type',
 'O',
 'L-date',
 'B-date',
 'U-date',
 'B-weather_type',
 'U-location',
 'L-location',
 'L-weather_type']